### Frames where Target Face is visible

In [ ]:
## Extracting Frames of Target Face

### Frames where Target Voice is audible

In [ ]:
## Exracting Frames of Target Voice

# Extracting Features

### Audio

In [1]:
from transformers import AutoModelForAudioClassification, AutoProcessor, pipeline
import torch
import librosa

# pipe = pipeline("audio-classification", model="ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition")     
pipe = pipeline("audio-classification", model="Hatman/audio-emotion-detection")     
# Load your audio file using librosa
audio_file = "audio.wav"
audio, rate = librosa.load(audio_file, sr=16000)  # Ensure it's resampled to 16kHz

# Get the emotion prediction
result = pipe(audio)

# Print the result
print(result)


2024-11-26 15:11:18.192960: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-26 15:11:18.213193: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-26 15:11:18.567655: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-26 15:11:20.939367: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Could not load the `decoder` for Hatman/audio-emotion-detection. Defaulting to raw CTC. Error: No module named 'ke

[{'score': 0.5844987034797668, 'label': 'Happy'}, {'score': 0.19212304055690765, 'label': 'Angry'}, {'score': 0.17521733045578003, 'label': 'Disgusted'}, {'score': 0.03364561125636101, 'label': 'Fearful'}, {'score': 0.0062223924323916435, 'label': 'Suprised'}]


### Text

In [15]:
# Whisper for text transcription
# Detecting Feeling in Text

import whisper
from transformers import pipeline

# Schritt 1: Audio transkribieren
def transcribe_audio(audio_path):
    # Whisper-Modell laden
    model = whisper.load_model("base")  # Du kannst auch "small", "medium", "large" wählen
    result = model.transcribe(audio_path)
    return result["text"]

# Schritt 2: Emotionen im Text analysieren
def analyze_emotion(text):
    # Emotionserkennungsmodell laden
    emotion_pipeline = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
    emotions = emotion_pipeline(text)
    return emotions

# Audio-Datei Pfad
audio_file = "audio.wav"

# Transkription
print("Transkribiere die Audiodatei...")
transcribed_text = transcribe_audio(audio_file)
print(f"Transkribierter Text: {transcribed_text}")

# Emotionserkennung
print("Analysiere die Emotionen im Text...")
emotions = analyze_emotion(transcribed_text)
print(emotions)
print("Erkannte Emotionen mit Scores:")
for emotion in emotions[0]:  # emotions[0] enthält die Liste der Emotionen
    print(f"{emotion['label']}: {emotion['score']:.2f}")


Transkribiere die Audiodatei...


  checkpoint = torch.load(fp, map_location=device)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Transkribierter Text:  Okay, sir, you agree with me. You don't actually have to go to Bronze Circle to get Google Maps to generate these images. All you have to do is go to the website.
Analysiere die Emotionen im Text...


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
  warnings.warn(



[[{'label': 'anger', 'score': 0.010466489009559155}, {'label': 'disgust', 'score': 0.010573158971965313}, {'label': 'fear', 'score': 0.0021569335367530584}, {'label': 'joy', 'score': 0.0030502486042678356}, {'label': 'neutral', 'score': 0.9656197428703308}, {'label': 'sadness', 'score': 0.0030216046143323183}, {'label': 'surprise', 'score': 0.0051116799004375935}]]
Erkannte Emotionen mit Scores:
anger: 0.01
disgust: 0.01
fear: 0.00
joy: 0.00
neutral: 0.97
sadness: 0.00
surprise: 0.01


### Video

In [4]:
import cv2
from fer import FER
from collections import Counter

def detect_emotions_in_video(video_path):
    """
    Detect and count emotions in a video
    
    Args:
        video_path (str): Path to the input video file
    """
    # Initialize the FER detector
    emotion_detector = FER(mtcnn=True)
    
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)
    
    # Check if video opened successfully
    if not video_capture.isOpened():
        print("Error: Could not open video file")
        return None
    
    # Prepare to store emotion results
    emotion_counts = Counter()
    total_detected_frames = 0
    
    # Frame-by-frame emotion detection
    while True:
        # Read a frame from the video
        ret, frame = video_capture.read()
        
        # Break the loop if no more frames
        if not ret:
            break
        
        # Detect emotions in the frame
        detected_emotions = emotion_detector.detect_emotions(frame)
        print(detected_emotions)

        # Process detected emotions
        if detected_emotions:
            total_detected_frames += 1
            for emotion_result in detected_emotions:
                # Get dominant emotion
                emotions = emotion_result['emotions']
                dominant_emotion = max(emotions, key=emotions.get)
                emotion_counts[dominant_emotion] += 1
    
    # Release resources
    video_capture.release()
    
    # Print emotion occurrence summary
    print("\nEmotion Occurrence Summary:")
    for emotion, count in emotion_counts.items():
        percentage = (count / total_detected_frames) * 100 if total_detected_frames > 0 else 0
        print(f"{emotion}: {count} times ({percentage:.2f}%)")
    
    return dict(emotion_counts)

In [5]:
video_path = "input_video.mp4"
detect_emotions_in_video(video_path)

  state_dict = torch.load(state_dict_path)

  state_dict = torch.load(state_dict_path)

  state_dict = torch.load(state_dict_path)



[]
[]
[]
[]
[{'box': [546, 83, 166, 216], 'emotions': {'angry': 0.08, 'disgust': 0.0, 'fear': 0.04, 'happy': 0.07, 'sad': 0.04, 'surprise': 0.03, 'neutral': 0.74}}, {'box': [916, 154, 97, 130], 'emotions': {'angry': 0.2, 'disgust': 0.01, 'fear': 0.06, 'happy': 0.01, 'sad': 0.27, 'surprise': 0.0, 'neutral': 0.45}}, {'box': [781, 32, 76, 99], 'emotions': {'angry': 0.06, 'disgust': 0.0, 'fear': 0.04, 'happy': 0.73, 'sad': 0.04, 'surprise': 0.0, 'neutral': 0.11}}, {'box': [1119, 8, 74, 90], 'emotions': {'angry': 0.38, 'disgust': 0.0, 'fear': 0.2, 'happy': 0.02, 'sad': 0.29, 'surprise': 0.06, 'neutral': 0.05}}, {'box': [240, 75, 73, 91], 'emotions': {'angry': 0.02, 'disgust': 0.0, 'fear': 0.06, 'happy': 0.37, 'sad': 0.46, 'surprise': 0.01, 'neutral': 0.08}}]
[{'box': [548, 87, 166, 213], 'emotions': {'angry': 0.09, 'disgust': 0.0, 'fear': 0.05, 'happy': 0.03, 'sad': 0.04, 'surprise': 0.03, 'neutral': 0.76}}, {'box': [917, 149, 98, 135], 'emotions': {'angry': 0.13, 'disgust': 0.0, 'fear': 0.

{'neutral': 414,
 'happy': 415,
 'angry': 323,
 'sad': 296,
 'fear': 85,
 'surprise': 2}